In [2]:
!git clone https://github.com/FabioMMaia/fjsp_genetic_tabu.git

Cloning into 'fjsp_genetic_tabu'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), done.


In [5]:
# import os
# os.path.append(r'/content/fjsp_genetic_tabu/src')

In [6]:
import sys
sys.path.insert(1, "/content/fjsp_genetic_tabu/src")

In [30]:
import requests
from importlib import reload
import pandas as pd
import numpy as np

In [28]:
import read_fjsp_file, dataviz_fjsp
reload(read_fjsp_file)
reload(dataviz_fjsp)

<module 'dataviz_fjsp' from '/content/fjsp_genetic_tabu/src/dataviz_fjsp.py'>

In [17]:
txt = requests.get("https://www.ime.usp.br/~cris/fjs/benchmark/fmj/mfjs10").text

In [24]:
dt_times, descr = read_fjsp_file.parse_data(txt, return_descr=True)

In [27]:
descr

{'N': 48, 'A': 36, 'K': 8}

In [25]:
dt_times.head()

machine,Job,Ope,M0,M1,M2,M3,M4,M5,M6,M7
0,Job-1,O1_1,247.0,223.0,100.0,NaN,NaN,NaN,NaN,NaN
1,Job-1,O1_2,NaN,130.0,NaN,140.0,NaN,NaN,123.0,NaN
2,Job-1,O1_3,NaN,NaN,NaN,150.0,160.0,NaN,200.0,NaN
3,Job-1,O1_4,NaN,NaN,NaN,NaN,NaN,NaN,210.0,145.0
4,Job-2,O2_1,214.0,NaN,150.0,NaN,NaN,NaN,NaN,NaN


In [31]:
def pipeline_FJSP(data_time, OS, MS, verbose=False, plot=False):

  assert isinstance(data_time, pd.DataFrame), "data_time must be a pd.DataFrame containing the columns Job, Ope, Mi, M_i+1 ... corresponding to the time needed to perfome operation"
  assert isinstance(OS, list) and len(OS) == len(data_time['Ope']), 'OS must be de gene must be a list responsible for ordering having the same length as the number of operation'
  assert isinstance(MS, list) and len(OS) == len(data_time['Ope']), 'MS must be de gene must be a list responsible for machine assigment having the same length as the number of operation'

  # df = pd.DataFrame(data_time)
  df = data_time.copy()
  df.set_index(['Ope'], inplace=True)
  machines = {}
  M = 0
  new_start = 0

  for machine in df.columns[df.columns.str.startswith('M')]:
    m = Machine(machine, df[machine])
    machines[machine] = m
    M+=1

  df = pd.DataFrame(data_time)

  #Inicializa dataframe
  unique_jobs = df['Job'].unique().tolist()
  tasks = {i + 1: job for i, job in enumerate(unique_jobs)}
  # all_ope = df['Ope'].sort_values().tolist()
  all_ope = df['Ope'].tolist() # certificar se Oij esta na ordem correta
  executed_Op = []
  all_machines = df.columns[df.columns.str.startswith('M')].tolist()
  unavailable_times_machines = {key: [] for key in all_machines}
  output = pd.DataFrame()

  for i in range(0, len(OS)):
    row = {}
    row['Job'] = tasks[OS[i]]
    Oi = str(OS[i])
    Oj = str(len([x for x in OS[:i+1] if x in [OS[i]]]))
    Op = 'O' + Oi + '_' + Oj
    row['Ope'] = Op
    executed_Op.append(Op)

    if Op.split('_')[-1]!='1':
      previous_Op = Op.split('_')[0] + '_' + str(int(Op.split('_')[-1])-1)
    else:
      previous_Op=None

    row['previous_Ope'] = previous_Op
    machine =  'M' + str(MS[all_ope.index(Op)])
    row['Resource'] = machine
    time_ope_M = df.query('Ope==@Op')[machine].iloc[0]


    if time_ope_M != time_ope_M:
      print('Solução Não Factível')
      return None

    row['Time_Execution'] = time_ope_M

    if previous_Op is None:
      machines[machine].assign_operation(Op)

    else:
      for m in machines.keys():
        for Op_ in machines[m].operations.keys():
          if previous_Op == Op_:
            new_start = machines[m].operations[previous_Op]['End']
            machines[machine].assign_operation(Op, new_start)
            break

    row['Start'] =  machines[machine].operations[Op]['Start']
    row['Finish'] =  machines[machine].operations[Op]['End']

    output = pd.concat([output,pd.DataFrame(row, index=[0])], axis=0, ignore_index=True)
    if verbose:
      print(row)
      print(machines)
      print('------------#--------------')

  if plot:
    dataviz_fjsp.plot_gantt(output)

  makespan = output['Finish'].max()

  return output, makespan

In [33]:
import os
os.chdir('/content/fjsp_genetic_tabu')

In [34]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	src/

nothing added to commit but untracked files present (use "git add" to track)


In [35]:
!git add .

In [39]:
!git remote add origin https://<USERNAME>:<Token>@github.com/<USERNAME>/reponame.git


/bin/bash: line 1: USERNAME: No such file or directory
